In [1]:
%cd ..

/home/eli/AnacondaProjects/categorical_bpl


In [2]:
import argparse
import collections
import pyro
import torch
import numpy as np
import data_loader.data_loaders as module_data
import model.model as module_arch
from parse_config import ConfigParser
from trainer import Trainer

In [3]:
%matplotlib inline

In [4]:
# pyro.enable_validation(True)
# torch.autograd.set_detect_anomaly(True)

In [5]:
# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [6]:
Args = collections.namedtuple('Args', 'config resume device')
config = ConfigParser.from_args(Args(config='chemical_config.json', resume=None, device=None))

In [7]:
logger = config.get_logger('train')

In [8]:
# setup data_loader instances
data_loader = config.init_obj('data_loader', module_data)
valid_data_loader = data_loader.split_validation()

In [9]:
# build model architecture, then print to console
model = config.init_obj('arch', module_arch)

In [10]:
optimizer = pyro.optim.ReduceLROnPlateau({
    'optimizer': torch.optim.Adam,
    'optim_args': {
        "lr": 1e-4,
        "weight_decay": 0,
        "amsgrad": True
    },
    "patience": 5,
    "factor": 0.1,
    "verbose": True,
})

In [11]:
# optimizer = config.init_obj('optimizer', pyro.optim)

In [12]:
trainer = Trainer(model, [], optimizer, config=config,
                  data_loader=data_loader,
                  valid_data_loader=valid_data_loader,
                  lr_scheduler=optimizer, log_images=False, log_step=256)

In [ ]:
trainer.train()

Train Epoch: 1 [1/225000 (0%)] Loss: 30.482910
Train Epoch: 1 [257/225000 (0%)] Loss: 14.324219
Train Epoch: 1 [513/225000 (0%)] Loss: 8.355957
Train Epoch: 1 [769/225000 (0%)] Loss: 9.407227
Train Epoch: 1 [1025/225000 (0%)] Loss: 9.940918
Train Epoch: 1 [1281/225000 (1%)] Loss: 3.972656
Train Epoch: 1 [1537/225000 (1%)] Loss: 3.714355
Train Epoch: 1 [1793/225000 (1%)] Loss: 4.396484
Train Epoch: 1 [2049/225000 (1%)] Loss: 5.774414
Train Epoch: 1 [2305/225000 (1%)] Loss: 1.284668
Train Epoch: 1 [2561/225000 (1%)] Loss: 3.266602
Train Epoch: 1 [2817/225000 (1%)] Loss: 2.763672
Train Epoch: 1 [3073/225000 (1%)] Loss: 4.034180
Train Epoch: 1 [3329/225000 (1%)] Loss: 3.349121
Train Epoch: 1 [3585/225000 (2%)] Loss: 4.434570
Train Epoch: 1 [3841/225000 (2%)] Loss: 3.190430
Train Epoch: 1 [4097/225000 (2%)] Loss: 2.988281
Train Epoch: 1 [4353/225000 (2%)] Loss: 1.848633
Train Epoch: 1 [4609/225000 (2%)] Loss: 3.747559
Train Epoch: 1 [4865/225000 (2%)] Loss: 3.914062
Train Epoch: 1 [5121/225

In [ ]:
model.cpu()

In [ ]:
valid_xs, valid_ys = list(valid_data_loader)[-1]

In [ ]:
m, recons = model(observations=valid_xs, train=False)

In [ ]:
(recons == valid_xs).all(dim=-1).to(dtype=torch.float).mean(dim=0)